In [ ]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import load_model
import gradio as gr
import warnings
import os

warnings.filterwarnings("ignore")
try:
    import tensorflow as tf
    tf.get_logger().setLevel('ERROR')
    os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
except Exception:
    pass

# ---------------- CONFIG ----------------
CATEGORIES = ['Nature & Relaxation', 'Culture & Heritage', 'Shopping',
              'Food & Culinary', 'Hotels & Stay', 'Adventure']
CATEGORY_MAP = {cat: i for i, cat in enumerate(CATEGORIES)}
NUM_CATEGORIES = len(CATEGORIES)

MODEL_FILE = 'smart_tourism_dqn.h5'
DATA_FILE = 'india_complete_Full_tourism_dataset.csv'

# ---------------- LOAD ----------------
print("Loading dataset and model...")
try:
    df = pd.read_csv(DATA_FILE)
    df['Cost'] = pd.to_numeric(df['Cost'], errors='coerce')
    df['Rating'] = pd.to_numeric(df['Rating'], errors='coerce')
    df.dropna(subset=['Cost', 'Rating', 'Location', 'Category', 'Name'], inplace=True)
    df = df.reset_index(drop=True)
    print(f"✓ Dataset loaded: {len(df)} items")
except FileNotFoundError:
    print(f"✗ Error: '{DATA_FILE}' not found.")
    raise SystemExit(1)

try:
    model = load_model(MODEL_FILE)
    print(f"✓ Model loaded: {MODEL_FILE}\n")
except Exception as e:
    print(f"✗ Error: Could not load model '{MODEL_FILE}'. Error: {e}")
    raise SystemExit(1)

# ---------------- ENVIRONMENT ----------------
class SmartTourismEnv:
    def __init__(self, data, category_map):
        self.data = data
        self.category_map = category_map
        self.reset_episode_vars()

    def reset_episode_vars(self):
        self.package = []
        self.current_cost = 0.0
        self.selected_categories = set()
        self.location_items = pd.DataFrame()
        self.available_items = []

    def reset(self, budget, location, num_days, categories):
        self.budget = float(budget)
        self.initial_budget = float(budget)
        self.location = location
        self.num_days = int(num_days)
        self.required_categories = set(categories)
        self.num_required_categories = len(categories)

        self.reset_episode_vars()

        all_location_items = self.data[self.data['Location'] == location]
        if all_location_items.empty:
            return None

        self.location_items = all_location_items.reset_index(drop=True).copy()
        self.available_items = list(range(len(self.location_items)))
        self.action_space_size = len(self.location_items) + 1
        self.stop_action = len(self.location_items)
        return self._get_state()

    def _get_state(self):
        budget_used_ratio = self.current_cost / self.initial_budget if self.initial_budget > 0 else 0.0
        budget_remaining_ratio = (self.initial_budget - self.current_cost) / self.initial_budget if self.initial_budget > 0 else 0.0
        categories_covered = len(self.selected_categories)
        categories_coverage_ratio = categories_covered / self.num_required_categories if self.num_required_categories > 0 else 0.0

        required_vector = np.zeros(NUM_CATEGORIES)
        for cat in self.required_categories:
            if cat in self.category_map:
                required_vector[self.category_map[cat]] = 1

        selected_vector = np.zeros(NUM_CATEGORIES)
        for cat in self.selected_categories:
            if cat in self.category_map:
                selected_vector[self.category_map[cat]] = 1

        package_size = len(self.package)
        has_hotel = 1 if 'Hotels & Stay' in self.selected_categories else 0

        state = np.concatenate([
            [budget_used_ratio, budget_remaining_ratio, categories_coverage_ratio,
             package_size / 10.0, has_hotel, self.num_days / 10.0],
            required_vector,
            selected_vector
        ])

        return state.reshape(1, -1)

    def _calculate_hotel_cost(self, per_night_cost):
        per_night = float(per_night_cost)
        multiplied = per_night * self.num_days
        if multiplied <= self.budget:
            return multiplied
        if per_night <= self.budget:
            return per_night
        return multiplied

    def step(self, action):
        done = False

        if action == self.stop_action:
            return self._get_state(), 0, True, {}

        if action < 0 or action >= len(self.location_items) or action not in self.available_items:
            return self._get_state(), -1, False, {}

        item = self.location_items.iloc[action]
        item_category = item['Category']

        if item_category in self.selected_categories:
            return self._get_state(), -1, False, {}

        if str(item.get('Type', '')).strip().lower() == 'hotel' or item_category == 'Hotels & Stay':
            actual_cost = self._calculate_hotel_cost(item['Cost'])
        else:
            actual_cost = float(item['Cost'])

        if self.current_cost + actual_cost > self.budget:
            return self._get_state(), -1, False, {}

        self.available_items.remove(action)
        self.current_cost += actual_cost
        self.selected_categories.add(item_category)

        package_item = item.to_dict()
        package_item['ActualCost'] = actual_cost
        if str(item.get('Type', '')).strip().lower() == 'hotel' or item_category == 'Hotels & Stay':
            package_item['Nights'] = self.num_days
        self.package.append(package_item)

        affordable_items = []
        for i in self.available_items:
            cand = self.location_items.iloc[i]
            cand_cat = cand['Category']
            if cand_cat in self.selected_categories:
                continue
            if str(cand.get('Type', '')).strip().lower() == 'hotel' or cand_cat == 'Hotels & Stay':
                cand_cost = self._calculate_hotel_cost(cand['Cost'])
            else:
                cand_cost = float(cand['Cost'])
            if cand_cost <= (self.budget - self.current_cost):
                affordable_items.append(i)

        if not affordable_items:
            done = True

        return self._get_state(), 0, done, {}

# ---------------- PREDICTION ----------------
def predict_action(model, state, valid_actions, max_action_size):
    q_values = model.predict(state, verbose=0)[0]
    if len(q_values) < max_action_size:
        padded = np.full(max_action_size, -np.inf)
        padded[:len(q_values)] = q_values
        q_values = padded
    valid_q = [(a, q_values[a]) for a in valid_actions if a < len(q_values)]
    if not valid_q:
        return valid_actions[-1]
    return max(valid_q, key=lambda x: x[1])[0]

# ---------------- GREEDY POST-FILL ----------------
def find_best_affordable_for_category(env, category, prefer='cheapest'):
    candidates = []
    remaining = env.budget - env.current_cost
    for i in env.available_items:
        row = env.location_items.iloc[i]
        if row['Category'] != category:
            continue
        if category in env.selected_categories:
            continue
        if str(row.get('Type', '')).strip().lower() == 'hotel' or row['Category'] == 'Hotels & Stay':
            cost = env._calculate_hotel_cost(row['Cost'])
        else:
            cost = float(row['Cost'])
        if cost <= remaining:
            candidates.append((i, row, cost))
    if not candidates:
        return None
    if prefer == 'cheapest':
        candidates.sort(key=lambda t: (t[2], -float(t[1].get('Rating', 0.0))))
    else:
        candidates.sort(key=lambda t: (-float(t[1].get('Rating', 0.0)), t[2]))
    return candidates[0]

# ---------------- PACKAGE GENERATOR (Modified for GUI) ----------------
def generate_package_gui(location, budget, num_days, sort_by, sort_order, *category_checkboxes):
    """
    Generate package for Gradio interface with sorting
    """
    # Extract selected categories
    selected_categories = [CATEGORIES[i] for i, checked in enumerate(category_checkboxes) if checked]

    if not selected_categories:
        return None, "⚠️ Please select at least one category!", "", gr.update(visible=False)

    if not location or location.strip() == "":
        return None, "⚠️ Please select a location!", "", gr.update(visible=False)

    try:
        budget = float(budget)
        num_days = int(num_days)
        if budget <= 0 or num_days <= 0:
            raise ValueError
    except:
        return None, "⚠️ Please enter valid positive numbers for budget and days!", "", gr.update(visible=False)

    if location not in df['Location'].values:
        available_locs = ', '.join(sorted(df['Location'].unique())[:10])
        return None, f"❌ Location '{location}' not found!\n\nAvailable locations include: {available_locs}...", "", gr.update(visible=False)

    invalid = [c for c in selected_categories if c not in CATEGORIES]
    if invalid:
        return None, f"❌ Invalid categories: {', '.join(invalid)}", "", gr.update(visible=False)

    env = SmartTourismEnv(df, CATEGORY_MAP)
    state_or_msg = env.reset(budget, location, num_days, selected_categories)
    if state_or_msg is None:
        return None, f"❌ No items available for {location}", "", gr.update(visible=False)
    state = state_or_msg

    max_action_size = model.output_shape[-1]
    max_steps = 40

    for _ in range(max_steps):
        valid_actions = env.available_items + [env.stop_action]
        action = predict_action(model, state, valid_actions, max_action_size)
        if action not in valid_actions:
            action = env.stop_action
        next_state, _, done, _ = env.step(action)
        state = next_state
        if done:
            break

    # Greedy post-fill
    missing = [cat for cat in selected_categories if cat not in env.selected_categories]
    for cat in sorted(missing, key=lambda x: 0 if x == 'Hotels & Stay' else 1):
        found = find_best_affordable_for_category(env, cat, prefer='cheapest')
        if found:
            idx, row, cost = found
            env.available_items.remove(idx)
            env.current_cost += cost
            env.selected_categories.add(cat)
            pkg_item = row.to_dict()
            pkg_item['ActualCost'] = cost
            if cat == 'Hotels & Stay' or str(row.get('Type', '')).strip().lower() == 'hotel':
                pkg_item['Nights'] = env.num_days
            env.package.append(pkg_item)

    package = env.package

    if not package:
        return None, "❌ No suitable package found within your budget!\n\nTry:\n• Increasing your budget\n• Reducing number of days\n• Selecting fewer categories", "", gr.update(visible=False)

    # Sort package based on user selection
    if sort_by == "Cost":
        package.sort(key=lambda x: float(x.get('ActualCost', 0.0)), reverse=(sort_order == "Descending"))
    elif sort_by == "Rating":
        package.sort(key=lambda x: float(x.get('Rating', 0.0)), reverse=(sort_order == "Descending"))
    elif sort_by == "Category":
        package.sort(key=lambda x: str(x.get('Category', '')), reverse=(sort_order == "Descending"))

    # Build HTML for package
    package_html = f"""
    <div style='background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); padding: 40px; border-radius: 15px; margin-bottom: 20px; box-shadow: 0 8px 20px rgba(102, 126, 234, 0.3);'>
        <h1 style='color: white; text-align: center; margin: 0; font-size: 3em; font-weight: 700; text-shadow: 2px 2px 4px rgba(0,0,0,0.3);'>🎊 Your {location} Package</h1>
        <p style='color: white; text-align: center; font-size: 1.3em; margin-top: 15px; opacity: 1; font-weight: 500;'>
            ₹{int(budget):,} Budget | {int(num_days)} Day(s) | {len(package)} Activities
        </p>
    </div>

    <div style='background: white; border-radius: 10px; padding: 20px; box-shadow: 0 4px 15px rgba(0,0,0,0.1);'>
    """

    total_cost = 0.0
    for idx, item in enumerate(package, 1):
        name = str(item.get('Name', ''))
        category = str(item.get('Category', ''))
        rating = float(item.get('Rating', 0.0))
        cost = float(item.get('ActualCost', 0.0))
        total_cost += cost

        # Color coding by category
        category_colors = {
            'Nature & Relaxation': '#27ae60',
            'Culture & Heritage': '#e67e22',
            'Shopping': '#9b59b6',
            'Food & Culinary': '#e74c3c',
            'Hotels & Stay': '#3498db',
            'Adventure': '#f39c12'
        }
        color = category_colors.get(category, '#34495e')

        duration_text = ""
        if 'Nights' in item:
            duration_text = f"<span style='color: #7f8c8d;'>🌙 {int(item['Nights'])} night(s)</span>"

        package_html += f"""
        <div style='border-left: 4px solid {color}; padding: 15px; margin: 15px 0; background: linear-gradient(to right, {color}08, white); border-radius: 8px; transition: transform 0.2s; box-shadow: 0 2px 8px rgba(0,0,0,0.08);'>
            <div style='display: flex; justify-content: space-between; align-items: center;'>
                <div style='flex: 1;'>
                    <h3 style='margin: 0; color: {color}; font-size: 1.3em;'>{idx}. {name}</h3>
                    <p style='margin: 5px 0; color: #7f8c8d;'>
                        <span style='background: {color}; color: white; padding: 3px 10px; border-radius: 15px; font-size: 0.85em; font-weight: 600;'>{category}</span>
                        <span style='margin-left: 10px;'>⭐ {rating:.1f}/5.0</span>
                        {' | ' + duration_text if duration_text else ''}
                    </p>
                </div>
                <div style='text-align: right;'>
                    <p style='font-size: 1.5em; font-weight: bold; color: {color}; margin: 0;'>₹{cost:,.0f}</p>
                </div>
            </div>
        </div>
        """

    utilization = (total_cost/budget*100) if budget > 0 else 0
    remaining = budget - total_cost

    package_html += f"""
    </div>

    <div style='background: linear-gradient(135deg, #11998e 0%, #38ef7d 100%); padding: 25px; border-radius: 10px; margin-top: 20px; color: white; box-shadow: 0 6px 20px rgba(17, 153, 142, 0.3);'>
        <div style='display: grid; grid-template-columns: repeat(3, 1fr); gap: 20px; text-align: center;'>
            <div style='background: rgba(255,255,255,0.15); padding: 15px; border-radius: 10px; backdrop-filter: blur(10px);'>
                <p style='margin: 0; font-size: 0.9em; opacity: 0.9;'>Total Cost</p>
                <p style='margin: 5px 0 0 0; font-size: 2em; font-weight: bold; text-shadow: 2px 2px 4px rgba(0,0,0,0.2);'>₹{total_cost:,.0f}</p>
            </div>
            <div style='background: rgba(255,255,255,0.15); padding: 15px; border-radius: 10px; backdrop-filter: blur(10px);'>
                <p style='margin: 0; font-size: 0.9em; opacity: 0.9;'>Remaining</p>
                <p style='margin: 5px 0 0 0; font-size: 2em; font-weight: bold; text-shadow: 2px 2px 4px rgba(0,0,0,0.2);'>₹{remaining:,.0f}</p>
            </div>
            <div style='background: rgba(255,255,255,0.15); padding: 15px; border-radius: 10px; backdrop-filter: blur(10px);'>
                <p style='margin: 0; font-size: 0.9em; opacity: 0.9;'>Budget Used</p>
                <p style='margin: 5px 0 0 0; font-size: 2em; font-weight: bold; text-shadow: 2px 2px 4px rgba(0,0,0,0.2);'>{utilization:.1f}%</p>
            </div>
        </div>
    </div>
    """

    # Summary text - ONLY show what's actually covered
    avg_rating = np.mean([item.get('Rating', 0.0) for item in package])
    missing_after = [cat for cat in selected_categories if cat not in env.selected_categories]

    summary_html = f"""
    <div style='background: linear-gradient(135deg, #2c3e50 0%, #34495e 100%); border-radius: 10px; padding: 25px; box-shadow: 0 4px 15px rgba(0,0,0,0.2); margin-top: 20px; color: white;'>
        <h2 style='color: white; border-bottom: 3px solid #3498db; padding-bottom: 10px; margin-top: 0;'>📊 Package Summary</h2>

        <div style='background: rgba(255,255,255,0.1); border-radius: 8px; padding: 15px; margin: 10px 0;'>
            <p style='margin: 8px 0; font-size: 1.1em;'><strong>🎉 PACKAGE GENERATED SUCCESSFULLY!</strong></p>
        </div>

        <div style='background: rgba(255,255,255,0.1); border-radius: 8px; padding: 15px; margin: 10px 0;'>
            <p style='margin: 8px 0; font-size: 1.05em;'>📍 <strong>Destination:</strong> {location}</p>
            <p style='margin: 8px 0; font-size: 1.05em;'>💰 <strong>Total Cost:</strong> ₹{total_cost:,.0f}</p>
            <p style='margin: 8px 0; font-size: 1.05em;'>💵 <strong>Remaining Budget:</strong> ₹{remaining:,.0f}</p>
            <p style='margin: 8px 0; font-size: 1.05em;'>📊 <strong>Budget Utilization:</strong> {utilization:.1f}%</p>
        </div>

        <div style='background: rgba(255,255,255,0.1); border-radius: 8px; padding: 15px; margin: 10px 0;'>
            <p style='margin: 8px 0; font-size: 1.05em;'>📦 <strong>Total Items:</strong> {len(package)}</p>
            <p style='margin: 8px 0; font-size: 1.05em;'>⭐ <strong>Average Rating:</strong> {avg_rating:.2f}/5.0</p>
            <p style='margin: 8px 0; font-size: 1.05em;'>🎯 <strong>Categories Covered:</strong> {len(env.selected_categories)}/{len(selected_categories)}</p>
        </div>
    """

    if missing_after:
        summary_html += f"""
        <div style='background: rgba(231, 76, 60, 0.2); border-left: 4px solid #e74c3c; border-radius: 8px; padding: 15px; margin: 10px 0;'>
            <p style='margin: 0; font-size: 1.05em;'><strong>⚠️ Missing Categories:</strong> {', '.join(missing_after)} (due to budget constraints)</p>
        </div>
        """
    else:
        summary_html += """
        <div style='background: rgba(46, 204, 113, 0.2); border-left: 4px solid #2ecc71; border-radius: 8px; padding: 15px; margin: 10px 0;'>
            <p style='margin: 0; font-size: 1.05em;'><strong>✅ All requested categories included!</strong></p>
        </div>
        """

    summary_html += "</div>"

    # Suggestions - REDUCED TO 5 ITEMS
    other_items = df[df['Location'] == location].sort_values('Rating', ascending=False)
    seen = set()
    suggestions_html = """
    <div style='background: white; border-radius: 10px; padding: 20px; box-shadow: 0 4px 15px rgba(0,0,0,0.1); margin-top: 20px;'>
        <h2 style='color: #2c3e50; border-bottom: 3px solid #f39c12; padding-bottom: 10px; margin-top: 0;'>💡 Other Top-Rated Activities</h2>
    """

    shown = 0
    for _, r in other_items.iterrows():
        nm = str(r['Name'])
        if nm in seen or shown >= 5:  # REDUCED TO 5
            break
        seen.add(nm)

        cat = str(r['Category'])
        rating = float(r['Rating'])

        category_colors = {
            'Nature & Relaxation': '#27ae60',
            'Culture & Heritage': '#e67e22',
            'Shopping': '#9b59b6',
            'Food & Culinary': '#e74c3c',
            'Hotels & Stay': '#3498db',
            'Adventure': '#f39c12'
        }
        color = category_colors.get(cat, '#34495e')

        suggestions_html += f"""
        <div style='padding: 12px; margin: 10px 0; background: linear-gradient(to right, {color}08, white); border-radius: 8px; border-left: 4px solid {color}; box-shadow: 0 2px 8px rgba(0,0,0,0.06); transition: transform 0.2s;'>
            <div style='display: flex; justify-content: space-between; align-items: center;'>
                <div>
                    <strong style='color: {color}; font-size: 1.1em;'>{nm}</strong>
                    <span style='margin-left: 15px; background: {color}; color: white; padding: 2px 8px; border-radius: 10px; font-size: 0.85em; font-weight: 600;'>{cat}</span>
                </div>
                <span style='color: #f39c12; font-weight: bold; font-size: 1.1em;'>⭐ {rating:.1f}</span>
            </div>
        </div>
        """
        shown += 1

    suggestions_html += "</div>"

    return package_html, summary_html, suggestions_html, gr.update(visible=True)

# ---------------- GRADIO INTERFACE ----------------
def create_interface():
    # Get unique locations sorted
    locations = sorted(df['Location'].unique().tolist())

    with gr.Blocks(theme=gr.themes.Soft(), css="""
        .gradio-container {
            font-family: 'Inter', 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
            max-width: 1500px !important;
        }
        .gr-button-primary {
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%) !important;
            border: none !important;
            font-size: 1.2em !important;
            padding: 15px 30px !important;
            font-weight: 600 !important;
            box-shadow: 0 4px 15px rgba(102, 126, 234, 0.4) !important;
            transition: all 0.3s ease !important;
        }
        .gr-button-primary:hover {
            transform: translateY(-2px) !important;
            box-shadow: 0 6px 20px rgba(102, 126, 234, 0.6) !important;
        }
        .gr-button-secondary {
            background: linear-gradient(135deg, #f093fb 0%, #f5576c 100%) !important;
            border: none !important;
            font-weight: 600 !important;
            box-shadow: 0 3px 10px rgba(240, 147, 251, 0.3) !important;
        }
        .gr-button-secondary:hover {
            transform: translateY(-1px) !important;
            box-shadow: 0 5px 15px rgba(240, 147, 251, 0.5) !important;
        }
        .gr-form {
            background: linear-gradient(135deg, #e0e7ff 0%, #f3e7ff 100%) !important;
            border-radius: 15px !important;
            padding: 25px !important;
            border: none !important;
            box-shadow: 0 4px 15px rgba(0,0,0,0.08) !important;
        }
        h1 {
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            -webkit-background-clip: text;
            -webkit-text-fill-color: transparent;
            background-clip: text;
            font-size: 3em !important;
            font-weight: 800 !important;
            margin-bottom: 10px !important;
            text-align: center;
        }
        .gr-box {
            border-radius: 12px !important;
            box-shadow: 0 2px 10px rgba(0,0,0,0.05) !important;
        }
    """) as demo:

        gr.Markdown("""
        # 🌍 Smart Tourism Package Generator
        ### 🤖 AI-Powered Personalized Travel Experiences for Incredible India
        """)

        with gr.Row():
            # Left Panel - Input Section
            with gr.Column(scale=1):
                with gr.Group():
                    gr.Markdown("### 📋 Plan Your Perfect Trip")

                    location_dropdown = gr.Dropdown(
                        choices=locations,
                        label="🗺️ Select Your Destination",
                        info="Choose from India's top tourist locations",
                        interactive=True
                    )

                    with gr.Row():
                        budget_number = gr.Number(
                            label="💰 Your Budget (₹)",
                            value=50000,
                            minimum=5000,
                            maximum=1000000,
                            step=1000,
                            info="Enter your travel budget",
                            interactive=True
                        )

                        days_slider = gr.Slider(
                            minimum=1,
                            maximum=14,
                            value=3,
                            step=1,
                            label="📅 Trip Duration (Days)",
                            info="Number of days"
                        )

                with gr.Group():
                    gr.Markdown("### 🎯 Choose Your Travel Style")
                    gr.Markdown("*Select categories that match your interests*")

                    category_checkboxes = []
                    emoji_map = {
                        'Nature & Relaxation': '🌳',
                        'Culture & Heritage': '🏛️',
                        'Shopping': '🛍️',
                        'Food & Culinary': '🍽️',
                        'Hotels & Stay': '🏨',
                        'Adventure': '🏔️'
                    }

                    with gr.Row():
                        with gr.Column(scale=1):
                            for i in range(0, len(CATEGORIES), 2):
                                checkbox = gr.Checkbox(
                                    label=f"{emoji_map.get(CATEGORIES[i], '📍')} {CATEGORIES[i]}",
                                    value=False
                                )
                                category_checkboxes.append(checkbox)

                        with gr.Column(scale=1):
                            for i in range(1, len(CATEGORIES), 2):
                                checkbox = gr.Checkbox(
                                    label=f"{emoji_map.get(CATEGORIES[i], '📍')} {CATEGORIES[i]}",
                                    value=False
                                )
                                category_checkboxes.append(checkbox)

                with gr.Group():
                    gr.Markdown("### 🔄 Sort Package By")
                    with gr.Row():
                        sort_by = gr.Radio(
                            choices=["Default", "Cost", "Rating", "Category"],
                            value="Default",
                            label="Sort By",
                            info="Choose sorting criteria"
                        )
                        sort_order = gr.Radio(
                            choices=["Ascending", "Descending"],
                            value="Ascending",
                            label="Sort Order",
                            info="Order direction"
                        )

                with gr.Row():
                    generate_btn = gr.Button("🎁 Generate My Perfect Package", variant="primary", size="lg", scale=3)
                    clear_btn = gr.Button("🔄 Reset All", variant="secondary", size="lg", scale=1)

            # Right Panel - Results Section
            with gr.Column(scale=2):
                status_message = gr.Textbox(
                    label="📢 Status",
                    interactive=False,
                    lines=4,
                    visible=False
                )

                with gr.Group(visible=False) as results_group:
                    package_output = gr.HTML(label="")
                    summary_output = gr.HTML(label="")
                    suggestions_output = gr.HTML(label="")

        gr.Markdown("""
        ---
        ### 💡 Smart Tips for Best Results:

        🎯 **Category Selection:** Choose 2-4 categories for a well-balanced and diverse itinerary
        🏨 **Hotels & Stay:** Always include this for multi-day trips to ensure accommodation
        💰 **Budget Guidelines:** Allocate ₹15,000-25,000 per day per person for comfortable travel
        📅 **Optimal Duration:** 3-5 days is ideal for exploring most Indian destinations thoroughly
        🔄 **Sorting Options:** Use sort features to prioritize by cost, rating, or category

        **⚡ Powered by Deep Q-Network (DQN) Reinforcement Learning AI Model**
        """)

        # Button actions
        def generate_and_show(*args):
            pkg_html, summary, sugg_html, update_dict = generate_package_gui(*args)

            if pkg_html is None:
                # Error case
                return (
                    summary,
                    gr.update(visible=True),
                    gr.update(visible=False),
                    None,
                    None,
                    None
                )
            else:
                # Success case
                return (
                    "",
                    gr.update(visible=False),
                    gr.update(visible=True),
                    pkg_html,
                    summary,
                    sugg_html
                )

        generate_btn.click(
            fn=generate_and_show,
            inputs=[location_dropdown, budget_number, days_slider, sort_by, sort_order] + category_checkboxes,
            outputs=[status_message, status_message, results_group, package_output, summary_output, suggestions_output]
        )

        def reset_form():
            return (
                None,  # location
                50000,  # budget
                3,      # days
                "Default",  # sort_by
                "Ascending",  # sort_order
                *[False] * len(CATEGORIES),  # all checkboxes unchecked
                "",     # status_message
                gr.update(visible=False),  # status_message visibility
                gr.update(visible=False),  # results_group visibility
                None,   # package_output
                None,   # summary_output
                None    # suggestions_output
            )

        clear_btn.click(
            fn=reset_form,
            inputs=[],
            outputs=[location_dropdown, budget_number, days_slider, sort_by, sort_order] +
                    category_checkboxes + [status_message, status_message, results_group,
                    package_output, summary_output, suggestions_output]
        )

    return demo

# Launch the app
if __name__ == "__main__":
    demo = create_interface()
    demo.launch(share=True, show_error=True)